In [1]:
import pandas as pd
import lxml
import numpy as np
import glob
from itertools import cycle
from faker import Faker
import random
import jupyter_black
from random import randint
import sqlite3
from datetime import datetime

jupyter_black.load()

### Read, parse, process and generate more data

In [2]:
# read questions status
questions_status = pd.read_csv("../data/raw/HalQuestions_StatusOnly_Test.csv")

questions_status.to_csv("../data/processed/questions.csv", index=False)
questions_status

,Example Question,Hal-Answers
0,What is the current status of my loan applicat...,True
1,When can I expect a decision on my loan applic...,True
2,What additional information do you need from m...,True
3,Can I check the status of my loan application ...,True
4,Are there any updates on the progress of my lo...,True
...,...,...
1058,Is there any feedback or comments on my loan a...,True
1059,How will I be notified about the status of my ...,True
1060,Are there any fees associated with processing ...,False
1061,Has a credit check been conducted for my loan ...,False


In [3]:
def read_xml(file_path: str) -> pd.DataFrame:
    # read file
    df = pd.read_xml(file_path).replace({None: np.nan})

    # set client stage
    df["Stage"] = df.iloc[0]["Stage"]

    # set datetime in date
    df["Date"] = pd.to_datetime(df["Date"])

    # return only last row
    return pd.DataFrame(df.iloc[-1]).T

In [4]:
# read all xml data, parse and save it processed
files_paths = glob.glob("../data/raw/*.xml")

client_df = pd.concat([read_xml(file_path) for file_path in files_paths])
client_df

,Stage,Status,Collateral,ID,DecisionType,Date,IsCurrent,Rate,Amount,Term,Tier,UserCode,PreApproval,Reasons,Stipulations,SpecialConsiderations,AuditData,OtherData
2,Final,Declined,NaN,678901.0,System Decision,2023-08-16 11:15:00-07:00,True,0.0,0.0,0.0,NaN,789.0,\n,\n,\n,\n,\n,\n
2,Reviewed,Approved,NaN,87654321.0,System Decision,2023-08-16 09:45:00-07:00,True,2.875,45000.0,72.0,B,123.0,\n,\n,\n,\n,\n,\n
2,Initial,Counter,NaN,678901.0,System Decision,2023-08-16 10:15:00-07:00,True,4.125,20000.0,48.0,C,456.0,\n,\n,\n,\n,\n,\n
2,Initial,Counter,NaN,654321.0,System Decision,2023-08-16 12:15:00-07:00,True,3.75,18000.0,60.0,B,567.0,\n,\n,\n,\n,\n,\n


### Generate fake user data

In [5]:
all_status = ["Declined", "Approved", "Counter"]

stage_status_dict = {"Declined": "Final", "Approved": "Reviewed", "Counter": "Initial"}


def generate_rate(status: str) -> float:
    if status == "Declined":
        return 0
    else:
        return random.uniform(2, 7)


def generate_amount(status: str) -> float:
    if status == "Declined":
        return 0
    else:
        return round(random.uniform(10000, 50000), 2)


def generate_term(status: str) -> float:
    if status == "Declined":
        return 0
    else:
        return random.uniform(30, 80)


def generate_tier(status: str) -> str:
    if status == "Declined":
        return np.nan
    else:
        return random.choice(["A", "B", "C"])


def generate_user_code() -> float:
    return random.randint(1, 1000)


fake = Faker()


def create_rows(num_of_rows=1):
    """Create a list of num_of_rows rows generated randomly"""

    fake_df_in_list = []

    for x in range(num_of_rows):
        status = random.choice(all_status)
        stage = stage_status_dict[status]

        row = {
            "Stage": stage,
            "Status": status,
            "Collateral": np.nan,
            "ID": x,
            "DecisionType": "System Decision",
            "Date": fake.date_time_between(start_date="-2y", end_date="now"),
            "IsCurrent": True,
            "Rate": generate_rate(status),
            "Amount": generate_amount(status),
            "Term": generate_term(status),
            "UserCode": generate_user_code(),
            "Tier": generate_tier(status),
            "PreApproval": "\n",
            "Reasons": "\n",
            "Stipulations": "\n",
            "SpecialConsiderations": "\n",
            "AuditData": "\n",
            "OtherData": "\n",
        }

        fake_df_in_list.append(row)

    return fake_df_in_list

In [8]:
generated_client_df = pd.DataFrame(
    data=create_rows(num_of_rows=5000), columns=client_df.columns
)
generated_client_df = generated_client_df.rename(columns={"Date": "ModificationDate"})
generated_client_df

,Stage,Status,Collateral,ID,DecisionType,ModificationDate,IsCurrent,Rate,Amount,Term,Tier,UserCode,PreApproval,Reasons,Stipulations,SpecialConsiderations,AuditData,OtherData
0,Final,Declined,NaN,0,System Decision,2023-02-05 23:25:08,True,0.000000,0.00,0.000000,NaN,122,\n,\n,\n,\n,\n,\n
1,Initial,Counter,NaN,1,System Decision,2023-03-08 15:52:04,True,3.315448,22465.30,36.924913,C,948,\n,\n,\n,\n,\n,\n
2,Initial,Counter,NaN,2,System Decision,2022-09-20 15:56:28,True,6.723774,28117.88,58.220949,B,925,\n,\n,\n,\n,\n,\n
3,Initial,Counter,NaN,3,System Decision,2022-08-20 20:57:39,True,3.182543,11480.50,68.047243,A,924,\n,\n,\n,\n,\n,\n
4,Final,Declined,NaN,4,System Decision,2022-12-15 22:25:57,True,0.000000,0.00,0.000000,NaN,128,\n,\n,\n,\n,\n,\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Final,Declined,NaN,4995,System Decision,2022-07-14 12:57:42,True,0.000000,0.00,0.000000,NaN,451,\n,\n,\n,\n,\n,\n
4996,Initial,Counter,NaN,4996,System Decision,2022-11-17 02:52:08,True,4.759073,32168.80,49.138374,C,575,\n,\n,\n,\n,\n,\n
4997,Reviewed,Approved,NaN,4997,System Decision,2022-07-02 09:29:03,True,6.983140,41741.14,42.569696,C,43,\n,\n,\n,\n,\n,\n
4998,Reviewed,Approved,NaN,4998,System Decision,2022-04-15 10:26:26,True,5.802603,37380.17,43.887189,A,681,\n,\n,\n,\n,\n,\n


In [9]:
client_df = pd.concat([client_df, generated_client_df]).reset_index()

In [10]:
client_df.to_parquet("../data/processed/clients.parquet", index=False)
client_df

,index,Stage,Status,Collateral,ID,DecisionType,Date,IsCurrent,Rate,Amount,Term,Tier,UserCode,PreApproval,Reasons,Stipulations,SpecialConsiderations,AuditData,OtherData,ModificationDate
0,2,Final,Declined,NaN,678901.0,System Decision,2023-08-16 11:15:00-07:00,True,0.0,0.0,0.0,NaN,789.0,\n,\n,\n,\n,\n,\n,NaT
1,2,Reviewed,Approved,NaN,87654321.0,System Decision,2023-08-16 09:45:00-07:00,True,2.875,45000.0,72.0,B,123.0,\n,\n,\n,\n,\n,\n,NaT
2,2,Initial,Counter,NaN,678901.0,System Decision,2023-08-16 10:15:00-07:00,True,4.125,20000.0,48.0,C,456.0,\n,\n,\n,\n,\n,\n,NaT
3,2,Initial,Counter,NaN,654321.0,System Decision,2023-08-16 12:15:00-07:00,True,3.75,18000.0,60.0,B,567.0,\n,\n,\n,\n,\n,\n,NaT
4,0,Final,Declined,NaN,0,System Decision,NaN,True,0.0,0.0,0.0,NaN,122,\n,\n,\n,\n,\n,\n,2023-02-05 23:25:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999,4995,Final,Declined,NaN,4995,System Decision,NaN,True,0.0,0.0,0.0,NaN,451,\n,\n,\n,\n,\n,\n,2022-07-14 12:57:42
5000,4996,Initial,Counter,NaN,4996,System Decision,NaN,True,4.759073,32168.8,49.138374,C,575,\n,\n,\n,\n,\n,\n,2022-11-17 02:52:08
5001,4997,Reviewed,Approved,NaN,4997,System Decision,NaN,True,6.98314,41741.14,42.569696,C,43,\n,\n,\n,\n,\n,\n,2022-07-02 09:29:03
5002,4998,Reviewed,Approved,NaN,4998,System Decision,NaN,True,5.802603,37380.17,43.887189,A,681,\n,\n,\n,\n,\n,\n,2022-04-15 10:26:26


### Add to sqlite database

In [15]:
conn = sqlite3.connect("../data/clutch_database.db")

In [16]:
questions_status.to_sql("questions", conn, index=False, if_exists="replace")

1063

In [17]:
# fix time format
client_df["Date"] = pd.to_datetime(
    client_df["Date"].astype(str).apply(lambda x: x[:11])
)

In [18]:
client_df.to_sql("clients", conn, index=False, if_exists="replace")

5004